In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [6]:
# importing data
df = pd.read_csv('../2.Downloading data/processed_high_impact_outages.csv', parse_dates=['Start', 'Finish'])
df['file_name'] = pd.to_datetime(df['file_name'], format='%d/%m/%Y', errors='coerce')
df = df.replace({r"\r|\n": " "}, regex=True)
df.columns = df.columns.str.replace(r'[\r\n]', ' ', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
df = df.drop(columns=['Status_Description_Market', 'Market_Notice_Code', 'Impact_Region'])
df = df[df['Unplanned?'] != 1]

df.head(5)

C:\Users\althi\AppData\Local\Temp\ipykernel_19396\2952905288.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('../2.Downloading data/processed_high_impact_outages.csv', parse_dates=['Start', 'Finish'])
C:\Users\althi\AppData\Local\Temp\ipykernel_19396\2952905288.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('../2.Downloading data/processed_high_impact_outages.csv', parse_dates=['Start', 'Finish'])


,Region,NSP,Start,Finish,Network Asset,Recall_Day_Hours,Recall_Night_Hours,Project Work?,Unplanned?,DNSP Aware?,Generator Aware?,Inter-Regional,Status_Description,Status_Code,Reason,Duration_Hours,Impact,file_name
0,VIC/ TAS,AusNet,2022-03-22 06:30:00,2022-03-23 15:00:00,Loy Yang Power Station - Basslink 500 kV Line,4,6,0,0,0,0,1,Planned,STLTP,Maintenance,33.6,Tasmania will be separated from the NEM due to...,2022-03-21
1,VIC,AusNet,2023-12-19 05:30:00,2023-12-19 16:00:00,Moorabool–Sydenham No.2 500 kV Line,2,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,1344.0,A credible contingency event during this plann...,2023-03-27
2,VIC,AusNet,2023-05-10 06:00:00,2023-05-10 18:30:00,Heywood - Tarrone 500 kV Line,2,0,0,0,1,1,1,Planned,SUBMIT,Maintenance,753.6,A credible contingency event during this plann...,2023-03-27
3,VIC,AusNet,2024-05-01 07:30:00,2024-05-30 16:00:00,Heywood - Mortlake 500 kV Line,1,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,705.6,<<NEW since the last notification>> A credibl...,2023-12-18
4,VIC,AusNet,2024-05-01 07:30:00,2024-05-30 16:00:00,Heywood - Mortlake 500 kV Line,1,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,705.6,A credible contingency event during this plann...,2024-01-02


In [15]:
# processing the duplicates

# identify columns that uniquely define an outage (not including 'file_name')
outage_cols = ['NSP', 'Start', 'Finish', 'Network Asset', 'Reason', 'Duration_Hours']

# Sort by 'file_name' so the latest (most recent) is on top
df_sorted = df.sort_values(by='file_name', ascending=False)
df_sorted.head()

,Region,NSP,Start,Finish,Network Asset,Recall_Day_Hours,Recall_Night_Hours,Project Work?,Unplanned?,DNSP Aware?,Generator Aware?,Inter-Regional,Status_Description,Status_Code,Reason,Duration_Hours,Impact,file_name
8071,VIC,AusNet,2024-11-11 06:00:00,2024-11-11 16:00:00,Haunted Gully - Tarrone 500 Kv line,2,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,10.0,A credible contingency event during this plann...,2024-08-26
8438,VIC,AusNet,2025-02-04 06:00:00,2025-02-04 16:00:00,Bulgana - Horsham 220 kV Line,60,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,10.0,A credible contingency event during this plann...,2024-08-26
5079,VIC,AusNet,2025-03-03 05:00:00,2025-03-03 17:00:00,Kiamal SF-Red Cliffs 220 kV LINE,3,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,12.0,A credible contingency event during this plann...,2024-08-26
8920,VIC,AusNet,2025-06-21 07:00:00,2025-06-21 17:00:00,Kiamal SF-Red Cliffs 220 kV LINE,60,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,10.0,A credible contingency event during this plann...,2024-08-26
8340,VIC,AusNet,2025-01-27 06:00:00,2025-01-27 16:00:00,MooraboolSydenham No.2 500 kV Line,4,0,0,0,0,0,0,Planned,SUBMIT,Maintenance,10.0,A credible contingency event during this plann...,2024-08-26


In [ ]:

# Keep only the first occurrence of each outage (i.e., the latest version)
df_deduplicated = df_sorted.drop_duplicates(subset=outage_cols, keep='first')

# Find the duplicate rows (older versions) and save them separately
df_duplicates = df_sorted[df_sorted.duplicated(subset=outage_cols, keep='first')]

In [14]:
print(df_deduplicated.shape)
print(df_duplicates.shape)

(2388, 18)
(16320, 18)


In [11]:
df_deduplicated.to_csv('unique_outages.csv', index=False)
df_duplicates.to_csv('duplicate_outages.csv', index=False)